# Getting Started Guide

## Using Coach as a Framework

When running Coach as a framework, we use a Preset module to define the experiment parameters.
The preset is essentially a python module which instantiates a GraphManager object with an agent, an environment, a running schedule and some visualization parameters.

Coach can be used from the command line with one of the predefined presets. For example:

`coach -p CartPole_DQN`

You can find all the predefined presets under the `presets` directory, or by listing them using the following command:

`coach -l`

Coach can also be used with an externally defined preset by passing the absolute path to the module and the name of the graph manager object which is defined in the preset: 

`coach -p /home/my_user/my_agent_dir/my_preset.py:graph_manager`

Some presets are generic for multiple environment levels, and therefore require defining the specific level through the command line:

`coach -p Atari_DQN -lvl breakout`

There are plenty of other command line arguments you can use in order to customize the experiment. A full documentation of the available arguments can be found using the following command:

`coach -h`

## Using Coach as a Library

Creating a very simple graph containing a single Clipped PPO agent running with the CartPole-v0 Gym environment:

In [ ]:
from rl_coach.agents.clipped_ppo_agent import ClippedPPOAgentParameters
from rl_coach.environments.gym_environment import GymVectorEnvironment
from rl_coach.graph_managers.basic_rl_graph_manager import BasicRLGraphManager
from rl_coach.graph_managers.graph_manager import SimpleSchedule

graph_manager = BasicRLGraphManager(
    agent_params=ClippedPPOAgentParameters(),
    env_params=GymVectorEnvironment(level='CartPole-v0'),
    schedule_params=SimpleSchedule()
)

Running the graph according to the given schedule:

In [ ]:
graph_manager.improve()

Running each phase manually:

In [ ]:
from rl_coach.core_types import EnvironmentSteps

graph_manager = BasicRLGraphManager(
    agent_params=ClippedPPOAgentParameters(),
    env_params=GymVectorEnvironment(level='CartPole-v0'),
    schedule_params=SimpleSchedule()
)

graph_manager.heatup(EnvironmentSteps(100))
graph_manager.train_and_act(EnvironmentSteps(100))

## Changing the default parameters

In [ ]:
from rl_coach.agents.clipped_ppo_agent import ClippedPPOAgentParameters
from rl_coach.environments.gym_environment import GymVectorEnvironment
from rl_coach.graph_managers.basic_rl_graph_manager import BasicRLGraphManager
from rl_coach.graph_managers.graph_manager import SimpleSchedule
from rl_coach.graph_managers.graph_manager import ScheduleParameters
from rl_coach.core_types import TrainingSteps, EnvironmentEpisodes, EnvironmentSteps

# schedule
schedule_params = ScheduleParameters()
schedule_params.improve_steps = TrainingSteps(10000000)
schedule_params.steps_between_evaluation_periods = EnvironmentSteps(2048)
schedule_params.evaluation_steps = EnvironmentEpisodes(5)
schedule_params.heatup_steps = EnvironmentSteps(0)

# agent parameters
agent_params = ClippedPPOAgentParameters()
agent_params.algorithm.discount = 1.0

graph_manager = BasicRLGraphManager(
    agent_params=agent_params,
    env_params=GymVectorEnvironment(level='CartPole-v0'),
    schedule_params=schedule_params
)

graph_manager.improve()


## Using a custom gym environment

We can use a custom gym environment without registering it. 
We just need the path to the environment module.
We can also pass custom parameters for the environment __init__

In [ ]:
from rl_coach.agents.clipped_ppo_agent import ClippedPPOAgentParameters
from rl_coach.environments.gym_environment import GymVectorEnvironment
from rl_coach.graph_managers.basic_rl_graph_manager import BasicRLGraphManager
from rl_coach.graph_managers.graph_manager import SimpleSchedule
from rl_coach.architectures.tensorflow_components.embedders.embedder import InputEmbedderParameters

# define the environment parameters
bit_length = 10
env_params = GymVectorEnvironment(level='rl_coach.environments.toy_problems.bit_flip:BitFlip')
env_params.additional_simulator_parameters = {'bit_length': bit_length, 'mean_zero': True}

# Clipped PPO
agent_params = ClippedPPOAgentParameters()
agent_params.network_wrappers['main'].input_embedders_parameters = {
    'state': InputEmbedderParameters(scheme=[]),
    'desired_goal': InputEmbedderParameters(scheme=[])
}

graph_manager = BasicRLGraphManager(
    agent_params=agent_params,
    env_params=env_params,
    schedule_params=SimpleSchedule()
)

In [ ]:
graph_manager.improve()

The path to the environment can also be set as an absolute path, as follows: `<absolute python module path>:<environment class>`. For example:

In [ ]:
env_params = GymVectorEnvironment(level='/home/user/my_environment_dir/my_environment_module.py:MyEnvironmentClass')